In [1]:
using Pkg
Pkg.activate(".")

Pkg.add("ArgParse")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Distributions")
Pkg.add("ProgressMeter")
Pkg.add("StaticArrays")
Pkg.add("Setfield")
Pkg.add("Tables")

  Activating project at `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per`
   Resolving package versions...
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Project.toml`
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Project.toml`
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Project.toml`
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Project.toml`
  No Changes to `~/Developer/Overdamped To Underdamped Langevin Limits/AB_C_X_per/Manifest.toml`
   Re

In [2]:
using LinearAlgebra, StaticArrays, Setfield
using Random, Distributions, Statistics
using ArgParse, ProgressMeter
using Tables, CSV, DataFrames

In [15]:
num_sim    = 10
domain_len = 20.0
time_step  = 1.0e-7

1.0e-7

In [16]:
# Parameters
p = (;
    Ω_inf=0.0,         # nm
    Ω_sup=domain_len,  # nm
    T=0.25,        # s
    Δt=time_step,   # s
    C₀=1.25e-4,     # nm^{-3}
    λ₁=40.5745,        # 1/s, varies based on Ω
    λ₀=17.3,           # 1/s
    ε=10.0,           # nm
    γ=0.5,            # ratio
    D=1.0e6           # nm
)
p = (; p...,
    δ=sqrt(2 * p.D * p.Δt),

    # update states 
    λ₁Δt=p.λ₁ * p.Δt,
    λ₀Δt=p.λ₀ * p.Δt
)

(Ω_inf = 0.0, Ω_sup = 20.0, T = 0.25, Δt = 1.0e-7, C₀ = 0.000125, λ₁ = 40.5745, λ₀ = 17.3, ε = 10.0, γ = 0.5, D = 1.0e6, δ = 0.4472135954999579, λ₁Δt = 4.05745e-6, λ₀Δt = 1.73e-6)

In [17]:
const DIM = 3  # dim of the domain

mutable struct Particle
    X::SVector{DIM,Float64}
end

mutable struct ReactState
    dir::Float64
    A::Particle
    B::Particle
    C::Particle
end

In [18]:
function period_dist(X1, X2, p)
    (; Ω_sup) = p
    diff1 = abs.(X1 - X2)
    diff2 = Ω_sup .- diff1
    return norm(min.(diff1, diff2))
end

period_dist (generic function with 1 method)

In [19]:
function init_states(num_states, p)
    (; Ω_sup) = p

    react_states = Vector{ReactState}(undef, num_states)
    for i in 1:num_states
        init_dir = 0.0

        init_X1 = Ω_sup * (@SVector rand(3))
        init_A = Particle(init_X1)

        init_X2 = Ω_sup * (@SVector rand(3))
        init_B = Particle(init_X2)

        init_X3 = Ω_sup * (@SVector rand(3))
        init_C = Particle(init_X3)

        init_state = ReactState(init_dir, init_A, init_B, init_C)
        react_states[i] = init_state
    end
    return react_states
end

init_states (generic function with 1 method)

In [20]:
function update_state(state, p)
    (; λ₁Δt, λ₀Δt, ε, Ω_sup) = p

    # Forward reaction-diffusion 
    if state.dir == 1.0
        # forward diffusion
        X1, X2 = forward_diff(state.A.X, state.B.X, p)

        if (rand() <= λ₁Δt) && (period_dist(X1, X2, p) <= ε)
            # forward reaction
            X3 = 0.5 * (X1 + X2)
            state.C.X = mod.(X3, Ω_sup)
            state.dir = 0.0
        else
            state.A.X = X1
            state.B.X = X2
        end

        # Backward reaction-diffusion
    else
        # backward diffusion
        X3 = backward_diff(state.C.X, p)

        if (rand() <= λ₀Δt)
            # backward reaction
            X1, X2 = backward_react(X3, p)
            state.A.X = X1
            state.B.X = X2
            state.dir = 1.0
        else
            state.C.X = X3
        end
    end
end

update_state (generic function with 1 method)

In [21]:
function count_C(react_states, num_states)
    num_AB = 0.0
    for i in 1:num_states
        num_AB += react_states[i].dir
    end
    return (num_states - num_AB)
end

count_C (generic function with 1 method)

In [22]:
function save_result(means, num_sim, p)
    (; Ω_sup, Δt, T) = p
    # Store
    df = DataFrame(mean=means)
    file_path = "results/ABCX_per_sim$num_sim" * "_Ω$Ω_sup" * "_Δt$Δt" * "_T$T.csv"
    CSV.write(file_path, df)
end

save_result (generic function with 1 method)

In [23]:
function forward_diff(X1, X2, p)
    # unpack variables from p 
    (; δ, Ω_sup) = p

    # generate standard normal random vector
    # store as static arrays
    Z₁ = @SVector randn(3)
    X1_next = X1 + δ * Z₁
    X1_next = mod.(X1_next, Ω_sup)

    Z₂ = @SVector randn(3)
    X2_next = X2 + δ * Z₂
    X2_next = mod.(X2_next, Ω_sup)

    return (X1_next, X2_next)
end

function backward_diff(X3, p)
    (; δ, Ω_sup) = p
    Z = @SVector randn(3)
    X3_next = X3 + δ * Z
    return mod.(X3_next, Ω_sup)
end

backward_diff (generic function with 1 method)

In [24]:
function backward_react(X3, p)
    (; ε, γ, Ω_sup) = p

    # displacement of positions
    # sample uniform rv in B(0,ε)
    Z = @SVector randn(3)
    U = rand()
    η = ε * cbrt(U) * (Z / norm(Z))

    # original equation
    # γ * X1 + (1-γ) * X2 = X3
    # X1 - X2 = η
    X1 = X3 + (1 - γ) * η
    X1 = mod.(X1, Ω_sup)

    X2 = X3 - γ * η
    X2 = mod.(X2, Ω_sup)

    return (X1, X2)
end

backward_react (generic function with 1 method)

In [25]:
(; T, Δt, C₀, Ω_sup) = p

num_Δt = Int(floor(T / Δt))
num_states = Int(C₀ * (Ω_sup)^3)

save_time_step = T / 5000
step_ratio = Int(floor(save_time_step / Δt))
num_save_steps = Int(floor(T / save_time_step)) + 1
num_C_sim_path = zeros(num_save_steps)
num_C_total = zeros(num_save_steps)

num_C_sim_path[1] = Float64(num_states)

# Main loop
@showprogress for n in 1:num_sim
    react_states = init_states(num_states, p)
    for i in 1:num_Δt
        for j in 1:num_states
            update_state(react_states[j], p)
        end
        if mod(i, step_ratio) == 0
            num_C_sim_path[div(i, step_ratio)+1] = count_C(react_states, num_states)
        end
    end
    @. num_C_total += num_C_sim_path
end
C_means = num_C_total / num_sim
save_result(C_means, num_sim, p)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


"results/ABCX_per_sim10_Ω20.0_Δt1.0e-7_T0.25.csv"